In [1]:
#@unpublished{SARC,
#  authors={Mikhail Khodak and Nikunj Saunshi and Kiran Vodrahalli},
#  title={A Large Self-Annotated Corpus for Sarcasm},
#  url={https://arxiv.org/abs/1704.05579},
#  year=2017
#}
#https://www.kaggle.com/danofer/sarcasm#train-balanced-sarcasm.csv
import os
import string
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sachin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sachin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# df - data frame
df = pd.read_csv('train-balanced-sarcasm.csv')
# dropping empty comment entries
df.dropna(subset=['comment'], inplace=True)

print('Type:')
print('Labels array ',type(df.label))
print('Comments array ',type(df.comment))
print('one comment line ', type(df.comment[0]))
print('Shape:')
print('Labels array ',df.label.shape)
print('Comments array ',df.comment.shape)
print('Two first entries:')
print (df.label[0], df.comment[0])
print (df.label[1], df.comment[1])

Type:
Labels array  <class 'pandas.core.series.Series'>
Comments array  <class 'pandas.core.series.Series'>
one comment line  <class 'str'>
Shape:
Labels array  (1010773,)
Comments array  (1010773,)
Two first entries:
0 NC and NH.
0 You do know west teams play against west teams more than east teams right?


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
# create the transform
tf_idf_vectorizer = TfidfVectorizer()
# tokenize and build vocab
T = tf_idf_vectorizer.fit(df.comment)
print(len(T.vocabulary_))

167435


In [4]:
# preprocess with nltk
def my_tokenizer(corpus):
    corpus_tokenized = []
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    sbs = nltk.stem.SnowballStemmer('english', ignore_stopwords=False)
    for comment in corpus:
        words = tokenizer.tokenize(comment)
        # Remove stop words
        stop_words = set(stopwords.words('english'))
        words = [word for word in words if not word in stop_words]
        
        cmnt_t = []
        for token in words:
            cmnt_t.append(sbs.stem(token))
            # make a string to be compatible with TfidfVectorizer
            c = ' '.join(cmnt_t)
        # Lemmitize
        #words = [lemmatizer.lemmatize(word) for word in words]
        
        corpus_tokenized.append(c)
    return corpus_tokenized

In [ ]:
df2 = my_tokenizer(df.comment)

In [5]:
for i in range(10):
    print(df.comment[i])
    print(df2[i], '\n')

NC and NH.


NameError: name 'df2' is not defined

In [ ]:
print(df2[0:5], '\n')
# convert to pandas Series type type to be compatible with TfidfVectorizer
df3 = pd.Series((v for v in df2))
print(type(df3), '\n')
print(type(df3[0]))
print(df3[0:5])

In [ ]:
tf_idf_vectorizer2 = TfidfVectorizer()
T2 = tf_idf_vectorizer2.fit(df3)
print(len(T2.vocabulary_))

167435 features from before now reduced to 131022 features, that is about 22%

In [17]:
from sklearn.model_selection import train_test_split
# divide into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df.comment, df.label, train_size=0.33)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [18]:
# Vetorize the training data set
X_train = tf_idf_vectorizer2.transform(X_train)
print(X_train.shape)
print(X_train[0:3])

NameError: name 'tf_idf_vectorizer2' is not defined

In [ ]:
# Vetorize the test data set
X_test = tf_idf_vectorizer.transform(X_test)
print(X_test.shape)
print(X_test[0:3])

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC

svc = SVC(C=1, kernel='rbf', gamma=0.01,cache_size=7000)
# bagging with Logistic Regressoin
# oob Out-Of-Bag
bag_log_reg = BaggingClassifier(
      svc, n_estimators=500,
    max_samples=50000, bootstrap=True, n_jobs=-1, oob_score=True)


bag_log_reg.fit(X_train,y_train)
print('oob_score is ', bag_log_reg.oob_score_)

NameError: name 'X_train' is not defined

In [ ]:
# 5-folds cross validation with grid search
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# I tried with n_jobs=-1 but kernel died after a few hours
Cs = [0.001, 0.01, 0.1, 1, 10]
gammas = [0.001, 0.01, 0.1, 1]
param_grid = {'C': Cs, 'gamma' : gammas}
grid_search = GridSearchCV(svm.SVC(kernel='rbf'), param_grid, cv=nfolds)


#clf = RandomForestClassifier(n_estimators=200, min_samples_leaf=50, random_state=31, n_jobs = -1)
grid_search.fit(X_train, y_train)
rint('oob_score is ', grid_search.oob_score_)

oob_score is  0.6572589228163271
